# Growth data analyses

In [69]:
# libraries
library(tidyverse)
library(multcomp)
library(ggpubr)
library(latex2exp)
#library(broom)
#library(purrr)
#library(car)
#library(data.table)
#library(formattable)
library(cowplot)

# Figure S1:

Here we present the results for the intial strains relative growth rate measurements:

In [50]:
# Read the data
ancestral =  read.csv("../../data/growth_ancestrals.csv")
head(ancestral,n=2)

,Treatment,Linage,Strain,Type,Experiment,Temperature,Max_deriv,Lag,k
,<fct>,<int>,<fct>,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>
1,Media,607,607P,Reference,18_3_18,43,0.001699398,477.6727,0.5504434
2,Media,607,607P,Reference,18_3_18,43,0.001868978,402.4976,0.5556532


In [51]:
# For each experiment
exp43 = ancestral %>% filter(Temperature == 43)
exp15 = ancestral %>% filter(Temperature == 15)
exp37 = ancestral %>% filter(Temperature == 37)

In [52]:
my_comparisons <- list(c("606P","607P") ,c("REL606","REL607"),c("606P","REL606"),c("607P","REL607"))

## Growth at $43^\circ{}C$

In [88]:
# Plot
#pdf("FigureS1_A.pdf")
ggboxplot(exp43, x = "Strain", y = "Max_deriv",
          color = "Strain", palette = "jco",add="jitter")+
stat_compare_means(comparisons = my_comparisons,method ="t.test",
                   method.args = list(alternative = "greater"),
                   p.adjust.method = "fdr",var.equal=F,
                   label="p.signif")+ylab("Maximum Growth Rate (OD/min)")+
 font("x.text", size = 11)+font("y.text", size = 11)
#dev.off()

pdf 
  2

## Growth at $15^\circ{}C$

In [89]:
# Plot
#pdf("FigureS1_C.pdf")
ggboxplot(exp15, x = "Strain", y = "Max_deriv",
          color = "Strain", palette = "jco",add="jitter")+
stat_compare_means(comparisons = my_comparisons,method ="t.test",p.adjust.method = "fdr",var.equal
=F,label="p.signif",method.args = list(alternative = "greater"))+ylab("Maximum Growth Rate (OD/min)")+ 
scale_y_continuous(labels = function(x) format(x, scientific = F))+
                    font("x.text", size = 11)+font("y.text", size = 11)
#dev.off()

pdf 
  2

## Growth at $37^\circ{}C$

In [86]:
# Plot
#pdf("FigureS1_B.pdf")
 ggboxplot(exp37, x = "Strain", y = "Max_deriv",
          color = "Strain", palette = "jco",add="jitter")+
stat_compare_means(comparisons = my_comparisons,method ="t.test",
                   method.args = list(alternative = "greater"),
                   p.adjust.method = "fdr",var.equal
=F,label="p.signif",)+ylab("Maximum Growth Rate (OD/min)")+ 
scale_y_continuous(labels = function(x) format(x, scientific = F)) +  
                   font("x.text", size = 11)+font("y.text", size = 11)
#dev.off()

pdf 
  2

# Table 1

In [ ]:
# Read the data
clones =  read.csv('../../data/growth_strains.csv')
head(clones,n=5)

In [ ]:
# Calculate the average
sum_clones = clones %>% group_by(Linage,Strain,Treatment,Experiment,Strain,Temperature,Type)  %>%
summarise(Mean = mean(Max_deriv), Std = sd(Max_deriv), Cv = sd(Max_deriv)/mean(Max_deriv), n= n() ) %>% ungroup()
sum_clones$Experiment = as.character(sum_clones$Experiment)
head(sum_clones)

In [ ]:
# Significant
clones = clones %>% distinct() %>% mutate(Group  = factor(paste(Strain,Experiment,Temperature,sep ="_")))
references = clones %>% filter(Type == "Reference")
cloners = clones %>% filter(Type == "Clone") %>% droplevels()
Strain = factor(unique(cloners$Group))
significant = data.frame(vars=double(),k=double(),Strain=factor(),Temperature=integer())
for (i in 1:length(factor(unique(cloners$Group)))){
    strain = Strain[i]
    test = cloners %>% filter(Group == strain)
    ref = references %>% filter(Experiment == test$Experiment[1] & Type == "Reference" & Linage ==  test$Linage[1])
    d = rbind(ref,test)
    # Sanity plot
    filename = paste(strain,test$Temperature[1],sep="_")
    pdf(filename)
    boxplot(d$Max_deriv~d$Type)
    dev.off()
    # t.test
    var_q = bartlett.test(Max_deriv~Type,d)
    var_T = (var_q$p.value>0.05)
    sig = d %>% summarise_each(funs(t.test(.[Type == "Clone"], .[Type == "Reference"],
                                           var.equal=var_T,alternative="greater")$p.value), 
                                           vars = Max_deriv,k)
    sig$Strain = test$Strain[1]
    sig$Temperature = test$Temperature[1]
    significant =  rbind(significant,sig)
    }
Growth = significant %>% dplyr::select(-k) %>% distinct() %>% spread(Temperature, vars)
Growth$Strain = factor(Growth$Strain, levels = c("R606-1", "R606-2", "R606-3", "R606-4","F606-1", "F606-2", "F606-3", "F606-4", "S606-1", "S606-2", "S606-3", "S606-4",
"R607-1", "R607-2", "R607-3", "R607-4","F607-1", "F607-2", "F607-3", "F607-4", "S607-1", "S607-2", "S607-3", "S607-4"))
Growth = Growth %>% arrange(Strain)
Growth

In [ ]:
# Replicate table 1
sum_clones$RelFit = 0
for (i in 1:nrow(sum_clones)){
    row_df = sum_clones[i,]
    reference =sum_clones %>% 
    filter(Experiment == as.character(row_df["Experiment"]) & 
           Type == "Reference" & 
           Temperature == as.numeric(row_df["Temperature"]) & 
           Linage == as.character(row_df["Linage"]) )
    sum_clones$RelFit[i] = row_df$Mean/reference$Mean
    
    
}
fitness_table = sum_clones %>% arrange(Linage, Temperature) %>% filter(Type == "Clone")
table1 = fitness_table %>% dplyr::select(Strain,Temperature,RelFit)  %>% distinct() %>% spread(Temperature, RelFit)
table1$Strain = factor(table1$Strain, levels = c("R606-1", "R606-2", "R606-3", "R606-4","F606-1", "F606-2", "F606-3", "F606-4", "S606-1", "S606-2", "S606-3", "S606-4",
"R607-1", "R607-2", "R607-3", "R607-4","F607-1", "F607-2", "F607-3", "F607-4", "S607-1", "S607-2", "S607-3", "S607-4"))
table1 = table1 %>% arrange(Strain)
table1 

In [ ]:
customGreen = "#71CA97"

customRed = "#ff7f7f"

In [ ]:
formattable(table1,`Strain`= color_tile(customGreen, customGreen0))

In [ ]:
library(pheatmap)

In [ ]:
lol = table1 %>% drop_na()
lol = as.matrix(lol[,2:3])

In [ ]:

pheatmap(lol)

In [ ]:
rownames(my.data) =my.data[,1]


In [ ]:
my.data = my.data[,2:3]

In [ ]:
x = 1:ncol(my.data)
y = 1:nrow(my.data)
centers <- expand.grid(y,x)
 
#make the plot margins a little bigger
par(mar = c(2,7,4,2))
 
image(x, y, t(my.data),
      col = c(rgb(0,0,1,0.3),rgb(1,0,0,0.3), rgb(1,1,0,0.3)),
      breaks = c(0, 25, 50, 100),
      xaxt = 'n', 
      yaxt = 'n', 
      xlab = '', 
      ylab = '',
      ylim = c(max(y) + 0.5, min(y) - 0.5)
      ) 

#add margin text
mtext(paste(attributes(my.data)$dimnames[[2]],"(%)"), at=1:ncol(my.data), padj = -1)
mtext(attributes(my.data)$dimnames[[1]], at=1:nrow(my.data), side = 2, las = 1, adj = 1.2)

#add black lines
abline(h=y + 0.5)
abline(v=x + 0.5)
